<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
<center>Автор материала: Куликов Павел Викторович, @kulikovpavel.

# <center>ELI5 - библиотека для визуализации и отладки ML моделей</center>


Ссылки:

[Документация](http://eli5.readthedocs.io/en/latest/) (отличная!)

[Github](https://github.com/TeamHG-Memex/eli5/blob/master/docs/source/index.rst)

Авторы: Михаил Коробов ([@kmike](https://opendatascience.slack.com/messages/@U064DRUF4)), Константин Лопухин ([@kostia](https://opendatascience.slack.com/team/U0P95857C))

[Мотивационное видео](https://www.youtube.com/watch?v=pqqcUzj3R90)

Установка

```pip install eli5```
> 

Библиотека из коробки умеет работать с линейными моделями, деревьями и ансамблями (scikit-learn, xgboost, LightGBM, lightning, sklearn-crfsuite) и в красивом виде показывает значимость признаков, может строить деревья, как текст или как картинки. Кроме этого есть важный функционал анализа предсказаний, можно визуально оценить, почему для того или иного примера ваша модель выдала тот или иной результат

![](https://raw.githubusercontent.com/TeamHG-Memex/eli5/master/docs/source/static/word-highlight.png)

Может работать с пайплайнами, в тот числе с HashingVectorizer и даже с препроцессингом в виде черного ящика, реализация алгоритма [LIME](https://arxiv.org/abs/1602.04938)

У библиотеки настолько прекрасная документация и подробные примеры, что просто проанализирую пару датасетов, а за остальным  лучше к ребятам на сайт


## XGBClassifier and LogisticRegression, categorial

Young People Survey. Explore the preferences, interests, habits, opinions, and fears of young people

[Ссылка на датасет](https://www.kaggle.com/miroslavsabo/young-people-survey)

In [ ]:
import eli5
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
df = pd.read_csv('responses.csv')

In [ ]:
df.head()

Возьмем в качестве целевой переменной место, где живет человек, деревня или город

In [ ]:
df['Village - town'].value_counts()

In [ ]:
df['Village - town'].fillna('city', inplace=True)

In [ ]:
X = df.drop(['Village - town'], axis=1)

In [ ]:
target = df['Village - town'].map(dict(city=0, village=1))

In [ ]:
import warnings

# xgboost <= 0.6a2 shows a warning when used with scikit-learn 0.18+
warnings.filterwarnings('ignore', category=DeprecationWarning)
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelBinarizer
from xgboost import XGBClassifier, XGBRegressor


# workaround for xgboost 0.7
def _check_booster_args(xgb, is_regression=None):
    # type: (Any, bool) -> Tuple[Booster, bool]
    if isinstance(xgb, eli5.xgboost.Booster): # patch (from "xgb, Booster")
        booster = xgb
    else:
        booster = xgb.get_booster() # patch (from "xgb.booster()" where `booster` is now a string)
        _is_regression = isinstance(xgb, XGBRegressor)
        if is_regression is not None and is_regression != _is_regression:
            raise ValueError(
                'Inconsistent is_regression={} passed. '
                'You don\'t have to pass it when using scikit-learn API'
                .format(is_regression))
        is_regression = _is_regression
    return booster, is_regression

eli5.xgboost._check_booster_args = _check_booster_args

In [ ]:
def prepare_df(data, columns=None):
    if not columns:
        columns = data.columns.values
        
    arr_categorial = list()
    
    for col in columns:
        lb = LabelBinarizer()
        transformed = lb.fit_transform(data[col].astype('str'))
        arr_categorial.append(pd.DataFrame(transformed, columns=col + '__' + lb.classes_.astype('object')).to_sparse())

    concated_df = pd.concat([data.drop(columns, axis=1)] + arr_categorial, axis=1).to_sparse()
    return concated_df

categorical_columns = ['Smoking', 'Alcohol', 'Punctuality', 'Lying', 'Internet usage', 'Gender', 'Left - right handed', 'Education', 'Only child', 'House - block of flats']
binarized_x = prepare_df(X, categorical_columns)

In [ ]:
xgb = XGBClassifier()

def evaluate(_clf, df, target):
    scores = cross_val_score(_clf, df, target, scoring='roc_auc', cv=10)
    print('Accuracy: {:.3f} ± {:.3f}'.format(np.mean(scores), 2 * np.std(scores)))
    _clf.fit(df, target)  # so that parts of the original pipeline are fitted

evaluate(xgb, binarized_x, target)

In [ ]:
eli5.explain_weights(xgb, top=50)

Важность признаков для классификатора. По умолчанию используется  прирост информации, "gain”, среднее значение по всем деревьям. Есть другие варианты, можно поменять через свойство importance_type.

Мы можем взглянуть теперь на конкретный пример

In [ ]:
eli5.show_prediction(xgb, binarized_x.iloc[300], show_feature_values=True)

Получили, что данный участник, вероятно, живет в городе, потому что не живет в квартире, тратит деньги на благотворительность и носит брендовые вещи

Посмотрим на логистическую регрессию

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

evaluate(lr, binarized_x.fillna('0'), target)

In [ ]:
eli5.show_weights(lr, feature_names=binarized_x.columns.values, top=100)

In [ ]:
eli5.show_prediction(lr, binarized_x.iloc[300].fillna('0'), show_feature_values=True)

Сразу заметно, что мы допустили ошибку (не отскалировали величины), и логистическая регрессия напрасно берет вес и рост как сильный значимый фактор, причем вес в плюс, а рост в минус, по сути компенсируя взаимно (факторы скоррелированы). И возраст тоже. Переобучение.

## Анализ текста

First GOP Debate Twitter Sentiment. Analyze tweets on the first 2016 GOP Presidential Debate

[Ссылка на датасет](https://www.kaggle.com/crowdflower/first-gop-debate-twitter-sentiment)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import make_pipeline

df = pd.read_csv("Sentiment.csv.zip")

In [ ]:
vec = CountVectorizer()
clf = LogisticRegressionCV()
pipe = make_pipeline(vec, clf)
pipe.fit(df.text, df.sentiment)

In [ ]:
eli5.show_weights(clf, vec=vec, top=20)

In [ ]:
eli5.show_prediction(clf, df.iloc[140].text, vec=vec)

In [ ]:
vec = TfidfVectorizer(analyzer='char_wb', ngram_range=(3,10), max_features=20000)
clf = LogisticRegressionCV()
pipe = make_pipeline(vec, clf)
pipe.fit(df.text, df.sentiment)

In [ ]:
eli5.show_weights(clf, vec=vec, top=20)

In [ ]:
eli5.show_prediction(clf, df.iloc[140].text, vec=vec)

При работе с большими объемами часто применятеся HashingVectorizer, для уменьшения размерности признакового пространства. ELI5 поддерживает работу с такими преобразованиями с помощью инвертирования.

```
from eli5.sklearn import InvertableHashingVectorizer
import numpy as np

vec = HashingVectorizer(stop_words='english', ngram_range=(1,2))
ivec = InvertableHashingVectorizer(vec)
sample_size = len(twenty_train.data) // 10
X_sample = np.random.choice(twenty_train.data, size=sample_size)
ivec.fit(X_sample);
```

http://eli5.readthedocs.io/en/latest/libraries/sklearn.html#reversing-hashing-trick

## LIME, черный ящик в текстовой обработке

Идея заключается в том, чтобы чуть-чуть менять входные строки, убирать случайным образом слова-символы, и смотреть как меняются предсказания модели, таким образом запоминать их влияние на на модель

http://eli5.readthedocs.io/en/latest/tutorials/black-box-text-classifiers.html


###

Павел Куликов

kulikovpavel@gmail.com

+7 903 118 37 41